In [1]:
import json, random, re, collections, itertools, base64, sys
from copy import deepcopy
from pathlib import Path
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from operator import itemgetter as at
from IPython.core.display import display, HTML
from ipywidgets import interact
display_html = lambda x: display(HTML(str(x)))
from pprint import pprint as pp
ls = lambda path: list(map(str, path.iterdir()))

sys.path.append('../src')
mturk_path = Path("../mturk/results")
data_path = Path("../data")
preprocessed_path = Path("../preprocessed")

%load_ext autoreload
%autoreload 2

In [2]:
ls(mturk_path)

['../mturk/results/Batch_4280306_batch_results.csv:Zone.Identifier',
 '../mturk/results/Batch_4258572_batch_results.csv',
 '../mturk/results/Batch_4280306_batch_results.csv']

In [3]:
def load_results(batch):
    df = pd.read_csv(str(next(mturk_path.rglob(f"*{batch}*.csv"))))
    df = df[[c for c in df.columns if "Answer." in c or 'Id' in c]]
    df = df.rename(columns={c:c.split('.',1)[1] for c in df.columns if '.' in c})
    return df.set_index("AssignmentId")
batch = 4280306
df=load_results(batch)
df

,HITId,HITTypeId,WorkerId,events,feedback,id,status
AssignmentId,,,,,,,
354GIDR5ZH1VMLMJ416WJOK9FKS00Z,32L724R86QF7UDAPZA5RF9PQESGIPI,3PUZG2R74XFTJN1AOYZP8I3IL1754O,A2VRDE2FHCBMF8,"[[{""start"":""2020-01-01T00:00:00"",""end"":""2020-0...",{},52446,1
3WMOAN2SRHSE0IF684WJFT6CH9FNVO,3511RHPAE09UWRYAX56TCKHXWEYLRR,3PUZG2R74XFTJN1AOYZP8I3IL1754O,A1KEA2Z47S3UPI,"[[{""start"":""2020-01-01T00:00:00"",""end"":""2020-0...",{},233689,1


In [4]:
 status_map = {
 "1": "All OK",
 "0": "All OK, not finished",
 "-1": "Missing Tool",
 "-2": "Missing Resource",
 "-3": "Missing Ingredient",
 "-4": "Redundant unused ingredient",
 "-5": "Missing Time Boundry",
 "-6": "Activity",
"-7": "Validation Error",
 "-10": "Other issue",
 }

In [5]:
html=""
for assignment, row in df.iterrows():
    idx = row["id"]
    status = status_map[str(row["status"])]
    html+=f'<a href="http://127.0.0.1:8080/annotate/{batch}/{assignment}" target="_new">{idx} - {status}</a><br />'
display_html(html)